# Pipeline de Preprocessing de Texte pour l'Analyse de Sentiments

Ce notebook contient un pipeline complet de preprocessing de texte optimisé pour l'analyse de commentaires Reddit. Le pipeline comprend plusieurs étapes de nettoyage, tokenisation, suppression des mots vides et lemmatisation.

## 📊 Chargement des Données

Cette section charge le dataset de commentaires Reddit ChatGPT et configure le répertoire pour les données NLTK.


In [74]:
import pandas as pd

ntlk_dir = 'nltk_data'
df = pd.read_csv('data/chatgpt-reddit-comments.csv')

## 🧹 Classe TextCleaner - Nettoyage du Texte

Cette classe effectue le nettoyage et la préprocessing du texte avec plusieurs étapes :

- **Suppression HTML** : Retire les balises HTML (`<p>`, `<div>`, etc.)
- **Conversion en minuscules** : Uniformise la casse
- **Suppression des URLs** : Retire les liens HTTP/HTTPS
- **Suppression des mentions/hashtags** : Retire @mentions et #hashtags
- **Suppression de la ponctuation** : Retire tous les caractères de ponctuation
- **Suppression des chiffres** : Retire les nombres
- **Normalisation des espaces** : Uniformise les espaces multiples

La méthode `clean_text()` applique toutes ces étapes en séquence.


In [75]:
import re
from bs4 import BeautifulSoup

class TextCleaner:
	"""
	Class for cleaning and preprocessing text data with multiple processing steps.
	"""
	
	def __init__(self) -> None:
		pass
	
	def remove_html(self, text) -> str:
		"""Remove HTML tags from text."""
		if not isinstance(text, str):
			return ""
		return BeautifulSoup(text, "html.parser").get_text()
	
	def convert_to_lowercase(self, text) -> str:
		"""Convert text to lowercase."""
		if not isinstance(text, str):
			return ""
		return text.lower()
	
	def remove_urls(self, text) -> str:
		"""Remove URLs from text."""
		if not isinstance(text, str):
			return ""
		return re.sub(r"http\S+", "", text)
	
	def remove_mentions_hashtags(self, text) -> str:
		"""Remove mentions (@username) and hashtags (#hashtag) from text."""
		if not isinstance(text, str):
			return ""
		return re.sub(r"@\w+|#\w+", "", text)
	
	def remove_punctuation(self, text) -> str:
		"""Remove punctuation from text."""
		if not isinstance(text, str):
			return ""
		return re.sub(r"[^\w\s]", "", text)
	
	def remove_digits(self, text) -> str:
		"""Remove digits from text."""
		if not isinstance(text, str):
			return ""
		return re.sub(r"\d+", "", text)
	
	def normalize_whitespace(self, text) -> str:
		"""Normalize whitespace in text."""
		if not isinstance(text, str):
			return ""
		return re.sub(r"\s+", " ", text).strip()
	
	def clean_text(self, text) -> str:
		"""
		Apply all cleaning steps to the text.
		
		Args:
			text (str): The text to clean
			
		Returns:
			str: The cleaned text
		"""
		if not isinstance(text, str):
			return ""
		
		# Apply all cleaning steps in sequence
		text = self.remove_html(text)
		text = self.convert_to_lowercase(text)
		text = self.remove_urls(text)
		text = self.remove_mentions_hashtags(text)
		text = self.remove_punctuation(text)
		text = self.remove_digits(text)
		text = self.normalize_whitespace(text)
		
		return text

cleaner = TextCleaner()

def clean_text(text) -> str:
	return cleaner.clean_text(text)


## 🔤 Classe TextTokenizer - Tokenisation

Cette classe divise le texte nettoyé en tokens (mots individuels) en utilisant le tokenizer de NLTK.

**Configuration NLTK** :
- Configure le chemin de données NLTK vers le dossier local
- Télécharge le package `punkt_tab` nécessaire pour la tokenisation

**Fonctionnalité** :
- Transforme une chaîne de caractères en liste de mots
- Gère les contractions et la ponctuation restante
- Retourne une liste vide si l'entrée n'est pas valide


In [76]:
import nltk
from nltk.tokenize import word_tokenize
import os

# Configure NLTK data path
nltk.data.path.append(os.path.abspath(ntlk_dir))

# Comment once done for the first time
# nltk.download('punkt_tab', download_dir=ntlk_dir)

class TextTokenizer:
	"""
	Class for tokenizing cleaned text into word-level tokens.
	"""
	
	def __init__(self) -> None:
		pass
	
	def tokenize(self, text) -> list[str]:
		"""
		Tokenize text into words.
		
		Args:
			text (str): The cleaned input text
		
		Returns:
			List[str]: List of tokens
		"""
		if not isinstance(text, str):
			return []
		return word_tokenize(text)

[nltk_data] Downloading package punkt_tab to nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


## 🚫 Classe StopwordRemover - Suppression des Mots Vides

Cette classe supprime les mots vides (stopwords) qui n'apportent pas de sens sémantique significatif.

**Caractéristiques** :
- Utilise la liste de stopwords anglais de NLTK
- **Préservation des négations** : Garde les mots comme "not", "no", "never", "n't" car ils sont cruciaux pour l'analyse de sentiment
- Paramètre de langue configurable (anglais par défaut)
- Comparaison insensible à la casse

**Objectif** : Réduire le bruit tout en préservant les indicateurs de sentiment importants.


In [77]:
import nltk
from nltk.corpus import stopwords

# Comment once done for the first time
# nltk.download('stopwords', download_dir=ntlk_dir)

class StopwordRemover:
	"""
	Class for removing stopwords from tokenized text.
	"""
	
	def __init__(self, language="english") -> None:
		"""
		Initialize the stopword remover with a given language.
		
		Args:
			language (str): Language of the stopwords (default is 'english')
		"""
		self.stop_words = set(stopwords.words(language))
	
	def remove_stopwords(self, tokens, keep_negation=True) -> list[str]:
		"""
		Remove stopwords from a list of tokens.
		
		Args:
			tokens (List[str]): List of word tokens
		
		Returns:
			List[str]: Tokens without stopwords
		"""
		if not isinstance(tokens, list):
			return []
		if keep_negation:
			negations = {"not", "no", "never", "n't"}
			return [word for word in tokens if word.lower() not in self.stop_words or word.lower() in negations]
		else:
			return [word for word in tokens if word.lower() not in self.stop_words]

[nltk_data] Downloading package stopwords to nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 🔄 Classe TextLemmatizer - Lemmatisation

Cette classe réduit les mots à leur forme canonique (lemme) pour normaliser les variations morphologiques.

**Packages NLTK requis** :
- `wordnet` : Base de données lexicale
- `omw-1.4` : Open Multilingual Wordnet
- `averaged_perceptron_tagger_eng` : Étiqueteur morpho-syntaxique anglais

**Fonctionnement** :
1. **POS Tagging** : Détermine la catégorie grammaticale de chaque mot
2. **Conversion des tags** : Convertit les tags TreeBank vers le format WordNet
3. **Lemmatisation** : Applique la lemmatisation en fonction du type grammatical

**Exemples** : "running" → "run", "better" → "good", "children" → "child"


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

# Comment once done for the first time
# nltk.download('wordnet', download_dir=ntlk_dir)
# nltk.download('omw-1.4', download_dir=ntlk_dir)
# nltk.download('averaged_perceptron_tagger_eng', download_dir=ntlk_dir)

class TextLemmatizer:
	"""
	Class for lemmatizing word tokens using NLTK's WordNetLemmatizer.
	"""
	
	def __init__(self) -> None:
		self.lemmatizer = WordNetLemmatizer()
	
	def get_wordnet_pos(self, treebank_tag) -> str:
		"""
		Convert POS tag from Treebank to WordNet format for better lemmatization.
		"""
		if treebank_tag.startswith('J'):
			return wordnet.ADJ
		elif treebank_tag.startswith('V'):
			return wordnet.VERB
		elif treebank_tag.startswith('N'):
			return wordnet.NOUN
		elif treebank_tag.startswith('R'):
			return wordnet.ADV
		else:
			return wordnet.NOUN  # fallback
	
	def lemmatize(self, tokens) -> list[str]:
		"""
		Lemmatize a list of word tokens.
		
		Args:
			tokens (List[str]): List of word tokens
		
		Returns:
			List[str]: Lemmatized tokens
		"""
		if not isinstance(tokens, list):
			return []

		pos_tags = nltk.pos_tag(tokens)  # POS tagging
		return [
			self.lemmatizer.lemmatize(token, self.get_wordnet_pos(pos))
			for token, pos in pos_tags
		]


[nltk_data] Downloading package wordnet to nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


## 🧪 Démonstration du Pipeline Complet

Cette section teste le pipeline complet sur des exemples variés pour illustrer chaque étape du preprocessing.

**Exemples de test** :
1. **HTML + Mentions/URLs** : Texte avec balises, mentions et liens
2. **Expressions informelles** : Contractions et ponctuation excessive  
3. **Négations** : Test de préservation des mots négatifs
4. **Contenu web** : Hashtags et URLs de sites web
5. **Formes grammaticales** : Différentes conjugaisons et pluriels

**Étapes visualisées** :
- Texte original → Texte nettoyé → Tokens → Sans stopwords → Lemmatisé

Cela permet de vérifier l'efficacité de chaque composant du pipeline.


In [79]:
# Exemple d'utilisation avec des chaînes de caractères aléatoires

# Créer les instances des classes
text_cleaner = TextCleaner()
tokenizer = TextTokenizer()
stopword_remover = StopwordRemover()
lemmatizer = TextLemmatizer()

# Exemples de textes avec différents problèmes
sample_texts = [
	"<p>Hello @user123! Check out this amazing #AI tool: https://example.com/awesome-tool 🚀</p>",
	"I'm loving the new ChatGPT updates!!! It's so much better than before... 😍",
	"Why are people still using OLD technologies in 2024??? Makes NO sense to me!!!",
	"<div>Visit our website www.example.com for more info about #MachineLearning and #DataScience</div>",
	"Running, jumped, better, good, children, mice, feet - testing different word forms"
]

print("=== DÉMONSTRATION DU PREPROCESSING ===\n")

for i, text in enumerate(sample_texts, 1):
	print(f"📝 Exemple {i}:")
	print(f"Original: {text}")
	
	# Étape 1: Nettoyage
	cleaned = text_cleaner.clean_text(text)
	print(f"Nettoyé: '{cleaned}'")
	
	# Étape 2: Tokenisation
	tokens = tokenizer.tokenize(cleaned)
	print(f"Tokens: {tokens}")
	
	# Étape 3: Suppression des mots vides
	tokens_no_stop = stopword_remover.remove_stopwords(tokens)
	print(f"Sans mots vides: {tokens_no_stop}")
	
	# Étape 4: Lemmatisation
	lemmatized = lemmatizer.lemmatize(tokens_no_stop)
	print(f"Lemmatisé: {lemmatized}")
	
	print("-" * 80 + "\n")


=== DÉMONSTRATION DU PREPROCESSING ===

📝 Exemple 1:
Original: <p>Hello @user123! Check out this amazing #AI tool: https://example.com/awesome-tool 🚀</p>
Nettoyé: 'hello check out this amazing tool'
Tokens: ['hello', 'check', 'out', 'this', 'amazing', 'tool']
Sans mots vides: ['hello', 'check', 'amazing', 'tool']
Lemmatisé: ['hello', 'check', 'amaze', 'tool']
--------------------------------------------------------------------------------

📝 Exemple 2:
Original: I'm loving the new ChatGPT updates!!! It's so much better than before... 😍
Nettoyé: 'im loving the new chatgpt updates its so much better than before'
Tokens: ['im', 'loving', 'the', 'new', 'chatgpt', 'updates', 'its', 'so', 'much', 'better', 'than', 'before']
Sans mots vides: ['im', 'loving', 'new', 'chatgpt', 'updates', 'much', 'better']
Lemmatisé: ['im', 'love', 'new', 'chatgpt', 'update', 'much', 'good']
--------------------------------------------------------------------------------

📝 Exemple 3:
Original: Why are people s